# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

****

## Part II: Data Cleaning

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest
from datetime import datetime

In [2]:
# RUN ONLY ONCE
# os.chdir is used to change the current directory to the specified path
os.chdir("../") # Sets path to the repo folder as it is one level above where this file exists!
path = os.getcwd()
print(path)

/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024


***

## Import Preprocessed Datasets

In [3]:
# Read Items_List.csv
Items = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv"))
Items.dtypes

ItemId             object
Description        object
CaseQty           float64
CaseUOM            object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [4]:
# Display first 5 rows
Items.head()

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-7631,5 SPICE POWDER,1.0,ea,1.0,lb,SPICES
1,I-3617,ALLSPICE GROUND SHAKER,8.0,each,454.0,g,SPICES
2,I-44059,ALMOND BUTTER*,10.0,Kg,1.0,Kg,FOOD - GROCERY
3,I-43652,ANCHO CHILLI,1.2,Kg,1.0,Kg,FOOD - GROCERY
4,I-2853,ANCHOVIES FILLET*,1.7,Kg,1.0,Kg,FOOD - GROCERY


In [5]:
# There are 486 rows and 7 columns
Items.shape

(667, 7)

In [6]:
# Read Ingredients_List.csv
Ingredients = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv"))
Ingredients.dtypes

IngredientId     object
Qty             float64
Uom              object
Conversion      float64
InvFactor       float64
Recipe           object
dtype: object

In [7]:
# Display first 5 rows
Ingredients.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-48933,1.00,Kg,1.00000,1.0000,P-10113
1,P-18746,1.00,Kg,1.00000,1.0000,P-10241
2,I-3388,1.00,L,1.00000,0.3058,P-10496
3,I-4658,2.27,Kg,2.20462,0.6942,P-10496
4,I-12176,4.00,g,0.00100,0.3636,P-12112


In [8]:
# There are 3278 rows and 6 columns
Ingredients.shape

(5112, 6)

In [9]:
# Read Preps_List.csv
Preps = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Preps_List.csv"))
Preps.dtypes

PrepId             object
Description        object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [10]:
# Display first 5 rows
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-61322,AIOLI|Pesto,4.000,L,ZDONT USE OK - PREP
1,P-26184,BAKED|Beans,9.000,Kg,PREP
2,P-54644,BAKED|Moussaka,16.000,PTN,NaN
3,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,NaN
4,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,NaN


In [11]:
# There are 546 rows and 5 columns
Preps.shape

(776, 5)

In [12]:
# Read Product_List.csv that was created from 1_data preprocessing
Products = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv"))
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [13]:
# Display first 5 rows
Products.head()

,ProdId,Description,SalesGroup
0,R-61778,ALF|Flatbread|4 Cheese,OK - AL FORNO
1,R-61780,ALF|Flatbread|Apple & Pancetta,OK - AL FORNO
2,R-61749,ALF|Flatbread|BBQ Chicken,OK - AL FORNO
3,R-61737,ALF|Flatbread|Beef & Onion,OK - AL FORNO
4,R-68122,ALF|Flatbread|Beef Onion|TMRW,OK - AL FORNO


In [14]:
# There are 223 rows and 3 columns
Products.shape

(342, 3)

In [15]:
# Read Coversions_List.csv that was created from 1_data preprocessing
Conversions = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv"))
Conversions.dtypes

ConversionId       object
Multiplier        float64
ConvertFromQty    float64
ConvertFromUom     object
ConvertToQty      float64
ConvertToUom       object
dtype: object

In [16]:
# Display first 5 rows
Conversions.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0,XXX,1.00,L
1,NaN,0.877193,1.0,1.14L,1.14,L
2,NaN,0.666667,1.0,1.5L,1.50,L
3,NaN,0.571429,1.0,1.75 L,1.75,L
4,NaN,0.500000,1.0,2L,2.00,L


In [17]:
# There are 270 rows and 6 columns
Conversions.shape

(310, 6)

***
## Update Conversion List

In [25]:
# Add the specific conversion info from the newly-processed data to a unit conversion database
Update_Conv = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Conv_UpdateConv.csv"))
Update_Conv

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-67659,0.008818,1.0,each,113.398,g
1,I-28697,0.005181,1.0,ea,193.000,g
2,I-47441,0.005181,1.0,ea,193.000,g
3,I-1905,0.012500,1.0,CT,80.000,g
4,I-47440,0.005051,1.0,ea,198.000,g
...,...,...,...,...,...,...
512,I-61327,0.002000,1.0,bottle,500.000,ml
513,I-64234,0.002415,1.0,414ML,414.000,ml
514,I-63697,0.008818,1.0,ea,113.400,g
515,I-13424,0.013333,2.0,CT,150.000,g


In [26]:
# return dataframe with null values
# There are no null values
subset_conv = Update_Conv[Update_Conv["Multiplier"].isna()]
subset_conv

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [27]:
# takes a data frame and assigns a new column called Multiplier to the dataframe
# uses iterrows() to iterate through the rows.Then subset_conv will have a new column named Multiplier with the computed values.
def assign_multiplier(df):
    for ind, row in df.iterrows():
        df.loc[ind, "Multiplier"] = row["ConvertFromQty"] / row["ConvertToQty"]
        
assign_multiplier(subset_conv)

# May 8th change: We want to 
assign_multiplier(Update_Conv)
#subset_conv

In [28]:
# convert dataframe to csv file
Update_Conv = pd.concat([Update_Conv, subset_conv], axis=0)
Update_Conv.to_csv("data/cleaning/update/Conv_UpdateConv.csv", index=False)

In [29]:
for index, row in Update_Conv.iterrows():
    Id = row['ConversionId'] 
    if Id in Conversions['ConversionId'].values:
        Conversions.drop(Conversions[Conversions['ConversionId'] == Id].index, inplace=True)
    else:
        print(f"Warning: 'ConversionId' {Id} not found in Conversions DataFrame. Skipping drop operation.")

In [30]:
frames = [Conversions, Update_Conv]
Conversions = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()

In [31]:
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0,XXX,1.00,L
1,NaN,0.877193,1.0,1.14L,1.14,L
2,NaN,0.666667,1.0,1.5L,1.50,L
3,NaN,0.571429,1.0,1.75 L,1.75,L
4,NaN,0.500000,1.0,2L,2.00,L
...,...,...,...,...,...,...
822,I-61327,0.002000,1.0,bottle,500.00,ml
823,I-64234,0.002415,1.0,414ML,414.00,ml
824,I-63697,0.008818,1.0,ea,113.40,g
825,I-13424,0.013333,2.0,CT,150.00,g


In [32]:
# Convert the DataFrame: Conversions into a csv file called Conversions_Added.csv
path = os.path.join(os.getcwd(), "data", "cleaning", "Conversions_Added.csv")
Conversions.to_csv(path, index = False, header = True)

### Create Unit Converter

In [33]:
# Import standard unit conversion information and construct a dataframe
Std_Unit = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "standard_conversions.csv"))
Std_Unit.head()

,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,4.92890,1,tsp,4.92890,ml
1,14.78700,1,Tbsp,14.78700,ml
2,946.35000,1,qt,946.35000,ml
3,473.17625,1,pt,473.17625,ml
4,28.34950,1,oz,28.34950,g


In [34]:
# Seperate uoms that converted to 'ml' or 'g'
liquid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'ml', 'ConvertFromUom'].tolist()
solid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'g', 'ConvertFromUom'].tolist()

In [35]:
# Construct a standard unit converter
def std_converter(qty, uom):
    if uom in Std_Unit['ConvertFromUom'].tolist():
        multiplier = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'Multiplier']
        Qty = float(qty)*float(multiplier)
        Uom = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'ConvertToUom'].values[0]
    else:
        Qty = qty
        Uom = uom
    return (Qty, Uom)

In [36]:
# Convert pounds to grams
std_converter(0.25,'lb')

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_92156/3899604546.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Qty = float(qty)*float(multiplier)


(113.398, 'g')

In [37]:
# Test the std_converter
assert std_converter(0.25,'lb') == (113.398, 'g')

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_92156/3899604546.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Qty = float(qty)*float(multiplier)


In [38]:
# Construct a unit converter for specific ingredients

# After this line below, spc_cov contains only the non-empty values from the 'ConversionId' column of the Conversions DataFrame.
spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))


# Comments for spc_converter:
# The function checks if ingredient is in the liquid_unit or solid_unit lists. If so, it calls std_converter(qty, uom) to 
# convert the quantity and UOM to a standardized unit.

# If uom is not in liquid_unit or solid_unit it checks if ingre is in spc_cov, if it is and the ConvertToUom is equal to grams 
# then the function applies the factor to the qty argument to convert it to the standardized unit, and returns the result as
# a tuple containing the converted quantity and uom. If no conversion found, then it calls std_converter(qty, uom)

# If uom not in liquid_unit or solid_unit and if ingre is not in spc_cov then the function calls std_converter(qty, uom)

def spc_converter(ingre, qty, uom):
    if uom in liquid_unit + solid_unit:
        return std_converter(qty, uom)
    elif ingre in spc_cov:
        conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
                                    & (Conversions['ConvertToUom'] == 'g')]
        multiplier = conversion['Multiplier']
        if multiplier.empty:
            return std_converter(qty, uom)
        else: 
            Qty = float(qty)/float(multiplier)
            Uom = conversion['ConvertToUom'].values[0]
            return (Qty, Uom)
    else:
        return std_converter(qty, uom)

In [39]:
# spc_cov2 is the same as spc_cov but without null values labelled as "nan" in the list
import math
spc_cov2 = [item for item in spc_cov if not(pd.isnull(item)) == True]
spc_cov2

['I-1120',
 'I-1197',
 'I-1232',
 'I-1813',
 'I-1813',
 'I-1820',
 'I-1820',
 'I-1821',
 'I-1821',
 'I-2074',
 'I-2074',
 'I-2159',
 'I-2159',
 'I-2159',
 'I-2217',
 'I-2220',
 'I-2220',
 'I-2220',
 'I-02378',
 'I-02378',
 'I-02378',
 'I-2254',
 'I-2254',
 'I-2306',
 'I-2310',
 'I-3113',
 'I-3113',
 'I-3113',
 'I-3141',
 'I-3143',
 'I-3143',
 'I-3144',
 'I-3144',
 'I-3144',
 'I-3145',
 'I-3145',
 'I-3146',
 'I-3146',
 'I-3149',
 'I-3159',
 'I-3223',
 'I-3223',
 'I-3225',
 'I-3225',
 'I-3284',
 'I-3284',
 'I-3284',
 'I-3321',
 'I-3321',
 'I-3342',
 'I-3348',
 'I-3356',
 'I-3365',
 'I-3365',
 'I-3367',
 'I-3370',
 'I-3386',
 'I-3386',
 'I-3386',
 'I-3387',
 'I-3387',
 'I-3387',
 'I-3387',
 'I-3388',
 'I-3388',
 'I-3388',
 'I-3389',
 'I-3389',
 'I-3389',
 'I-3390',
 'I-3390',
 'I-3390',
 'I-3391',
 'I-3391',
 'I-3391',
 'I-3392',
 'I-3392',
 'I-3392',
 'I-3435',
 'I-3435',
 'I-3449',
 'I-01293',
 'I-3468',
 'I-3497',
 'I-3545',
 'I-3545',
 'I-3545',
 'I-3572',
 'I-3572',
 'I-3575',
 'I-35

In [40]:
# Here we pass in the ingredient (I-1120) the quantity of the ingredient (1) and the unit of measurement: CT -> count
# This gives us the number of grams of that ingredient
spc_converter('I-1120', 1, 'CT')

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_92156/1590069188.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Qty = float(qty)/float(multiplier)


(50.0, 'g')

In [41]:
Conversions.loc[Conversions["ConversionId"] == "I-14190"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [42]:
c_list = Conversions["ConversionId"].unique()
"I-68700" in c_list

False

In [43]:
spc_converter("I-14190", 1, "LOAF")

(1, 'LOAF')

In [44]:
spc_converter('I-47530', 7, 'ea')

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_92156/1590069188.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Qty = float(qty)/float(multiplier)


(210.0, 'g')

***
## Items with Non-standard Units

In [45]:
Ingredients

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-48933,1.00,Kg,1.000000,1.0000,P-10113
1,P-18746,1.00,Kg,1.000000,1.0000,P-10241
2,I-3388,1.00,L,1.000000,0.3058,P-10496
3,I-4658,2.27,Kg,2.204620,0.6942,P-10496
4,I-12176,4.00,g,0.001000,0.3636,P-12112
...,...,...,...,...,...,...
5107,I-7787,50.00,ml,0.001000,1.0000,R-73430
5108,I-8281,50.00,g,0.001000,1.0000,R-73430
5109,P-38661,25.00,mg,0.000001,1.0000,R-73430
5110,P-73445,150.00,g,0.001000,1.0000,R-73452


In [46]:
Items

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-7631,5 SPICE POWDER,1.0,ea,1.0,lb,SPICES
1,I-3617,ALLSPICE GROUND SHAKER,8.0,each,454.0,g,SPICES
2,I-44059,ALMOND BUTTER*,10.0,Kg,1.0,Kg,FOOD - GROCERY
3,I-43652,ANCHO CHILLI,1.2,Kg,1.0,Kg,FOOD - GROCERY
4,I-2853,ANCHOVIES FILLET*,1.7,Kg,1.0,Kg,FOOD - GROCERY
...,...,...,...,...,...,...,...
662,I-72037,YOGURT YOGG*COCONUT VEGAN,3.5,Kg,1.0,Kg,DAIRY
663,I-68111,YOGURT YOGGU*PLAIN COCONUT,3.5,Kg,1.0,Kg,DAIRY
664,I-1490,ZEST FLAVOURS SUGAR ORANG,6.0,lb,1.0,lb,BAKING-RAW INGREDIENTS
665,I-1489,ZEST SUGARED LEMON,5.0,lb,1.0,lb,BAKING-RAW INGREDIENTS


In [47]:
# Filter out the items whose unit information is unknown 

# We find the column names
col_names = list(Ingredients.columns.values)

# Create a Items_Nonstd list
Items_Nonstd = []

# If the unit of measurement is not grams or ml and ingredient id starts with I and the ingredient is not in ConversionId column of Conversions 
# then we add it to Items_Nonstd list
for index, row in Ingredients.iterrows():
    Ingre = Ingredients.loc[index,'IngredientId']
    Uom = Ingredients.loc[index,'Uom']
    if Uom not in ['g', 'ml'] and Uom not in liquid_unit + solid_unit and Ingre.startswith('I') and Ingre not in Conversions["ConversionId"].tolist():
        Dict = {}
        Dict.update(dict(row))
        Items_Nonstd.append(Dict)

# Create a DataFrame from Items_Nonstd list
Items_Nonstd = pd.DataFrame(Items_Nonstd, columns = col_names)
# Remove duplicate ingredients of the same properties so that Items_Nonstd has only unique rows. 
Items_Nonstd.drop_duplicates(subset=['IngredientId'], inplace=True,)
Items_Nonstd

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-11842,2.000,LOAF,1.0,0.8818,P-13648
1,I-1028,0.500,CT,1.0,0.5000,P-26170
2,I-64492,1.000,LOAF,1.0,0.0625,P-26234
3,I-19735,1.000,CT,1.0,0.1000,P-26318
4,I-15358,50.000,ea,1.0,55.1268,P-26393
...,...,...,...,...,...,...
113,I-72414,0.025,cs,1.0,1.0000,R-73217
115,I-62928,1.000,ea,1.0,1.0000,R-73410
116,I-73417,1.000,ea,1.0,1.0000,R-73424
117,I-73414,1.000,ea,1.0,1.0000,R-73425


In [48]:
# Assigning a Description column to the Items_Nonstd    
for index, row in Items_Nonstd.iterrows():
    idx = row['IngredientId']
    filtered_items = Items.loc[Items['ItemId'] == idx, 'Description']
    if not filtered_items.empty:
        descrp = filtered_items.values[0]
        Items_Nonstd.loc[index, 'Description'] = descrp
    else:
        pass

In [49]:
Items_Nonstd.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe,Description
0,I-11842,2.0,LOAF,1.0,0.8818,P-13648,SOURDOUGH LOAFCOUNTRY
1,I-1028,0.5,CT,1.0,0.5000,P-26170,NAAN ORIGINAL*TEARDROP
2,I-64492,1.0,LOAF,1.0,0.0625,P-26234,LOAF GARLIC BREAD
3,I-19735,1.0,CT,1.0,0.1000,P-26318,PASTRY PUFF 10X15 IN
4,I-15358,50.0,ea,1.0,55.1268,P-26393,TORTILLA CORN*WHTE 6IN EX THN


In [50]:
# Convert the Items_Nonstd DataFrame to a csv file
path = os.path.join(os.getcwd(), "data", "cleaning", "Items_Nonstd.csv")
Items_Nonstd.to_csv(path, index = False, header = True)

***
## Clean Preps Units

In [51]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-61322,AIOLI|Pesto,4.000,L,ZDONT USE OK - PREP
1,P-26184,BAKED|Beans,9.000,Kg,PREP
2,P-54644,BAKED|Moussaka,16.000,PTN,NaN
3,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,NaN
4,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,NaN
...,...,...,...,...,...
771,P-46079,YIELD|Watermelon*,3.000,Kg,NaN
772,P-41047,YIELD|Yam,1.900,Kg,NaN
773,P-24452,YIELD|Yellow Onion,0.900,lb,NaN
774,P-57145,YIELD|Yellow Pepper,8.300,Kg,NaN


In [52]:
pd.read_csv("data/cleaning/update/Preps_UpdateUom.csv")

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.0,each,PREP,84.0,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.0,ea,PREP,24050.0,g
2,P-51992,YIELD|Bread|Sourdough 5/8,36.0,slice,NaN,1620.0,g
3,P-26234,BATCH|Roasted Garlic Bread,16.0,ea,PREP,1280.0,g
4,P-26170,GRILLED|NaanBread,1.0,ea,PREP,125.0,g
...,...,...,...,...,...,...,...
467,P-72874,BAKED|Cupcake Red Velvet,204.0,each,NaN,16396.0,g
468,P-4062,FF|Bowl|Thai Coconut DONTUSE,1.0,PTN,NaN,338.0,g
469,P-71996,PREP|smoothie|Strawb Banana,150.0,PTN,NaN,44700.0,g
470,P-73664,PREP|SMTHY|PressoEspresso|AAD,2.0,ea,NaN,582.5,g


In [53]:
# Creates 2 new columns called StdQty and StdUom in the Preps DataFrame. These columns contain NaN values
# Preparing to fill in these columns with standardized quantities and units of measurement 
Preps['StdQty'] = np.nan
Preps['StdUom'] = np.nan

In [54]:
# Convert uom into 'g' or 'ml' for each prep using the unit converter

# Retrieve the PrepId, PakQty, and PakUOM from the current row
# Pass these values to spc_converter, then we update the StdQty and StdUom columns of the current row with the converted values.
for index in Preps.index:
    PrepId = Preps.loc[index,'PrepId']
    Qty = Preps.loc[index,'PakQty']
    Uom = Preps.loc[index,'PakUOM']
    Preps.loc[index,'StdQty'] = spc_converter(PrepId, Qty, Uom)[0]
    Preps.loc[index,'StdUom'] = spc_converter(PrepId, Qty, Uom)[1]

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_92156/3899604546.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Qty = float(qty)*float(multiplier)
/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_92156/3899604546.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Qty = float(qty)*float(multiplier)
/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_92156/4107885356.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'ml' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Preps.loc[index,'StdUom'] = spc_converter(PrepId, Qty, Uom)[1]
/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_92156/3899604546.py:5: FutureWarning: Calling f

In [55]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-61322,AIOLI|Pesto,4.000,L,ZDONT USE OK - PREP,4000.0000,ml
1,P-26184,BAKED|Beans,9.000,Kg,PREP,9000.0000,g
2,P-54644,BAKED|Moussaka,16.000,PTN,NaN,16.0000,PTN
3,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,NaN,6176.0000,g
4,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,NaN,7360.0000,g
...,...,...,...,...,...,...,...
771,P-46079,YIELD|Watermelon*,3.000,Kg,NaN,3000.0000,g
772,P-41047,YIELD|Yam,1.900,Kg,NaN,1900.0000,g
773,P-24452,YIELD|Yellow Onion,0.900,lb,NaN,408.2328,g
774,P-57145,YIELD|Yellow Pepper,8.300,Kg,NaN,8300.0000,g


In [56]:
# Save cleaned preps list to file
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_Unit_Cleaned.csv")
Preps.to_csv(path, index = False, header = True)

In [57]:
pd.read_csv("data/cleaning/update/Preps_UpdateUom.csv")

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.0,each,PREP,84.0,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.0,ea,PREP,24050.0,g
2,P-51992,YIELD|Bread|Sourdough 5/8,36.0,slice,NaN,1620.0,g
3,P-26234,BATCH|Roasted Garlic Bread,16.0,ea,PREP,1280.0,g
4,P-26170,GRILLED|NaanBread,1.0,ea,PREP,125.0,g
...,...,...,...,...,...,...,...
467,P-72874,BAKED|Cupcake Red Velvet,204.0,each,NaN,16396.0,g
468,P-4062,FF|Bowl|Thai Coconut DONTUSE,1.0,PTN,NaN,338.0,g
469,P-71996,PREP|smoothie|Strawb Banana,150.0,PTN,NaN,44700.0,g
470,P-73664,PREP|SMTHY|PressoEspresso|AAD,2.0,ea,NaN,582.5,g


### Get Preps with Nonstandard Unit

In [58]:
col_names = list(Preps.columns.values)
Preps_Nonstd = []

for index, row in Preps.iterrows():
    StdUom = Preps.loc[index,'StdUom']
    if StdUom not in ['g', 'ml']:
        Dict = {}
        Dict.update(dict(row))
        Preps_Nonstd.append(Dict)

Preps_Nonstd = pd.DataFrame(Preps_Nonstd, columns = col_names)

In [59]:
Preps_Nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54644,BAKED|Moussaka,16.0,PTN,NaN,16.0,PTN
1,P-26318,BAKEDPuff Pastry,10.0,ea,ZDONT USE OK - PREP,10.0,ea
2,P-26234,BATCH|Roasted Garlic Bread,16.0,ea,PREP,16.0,ea
3,P-53361,BATTER|Pancake|Banana,24.0,ea,NaN,24.0,ea
4,P-35713,BLACKENED|Chicken,185.0,ea,PREP,185.0,ea
5,P-26047,BOILED|Hard Boiled Eggs FT,50.0,ea,PREP,50.0,ea
6,P-51518,COOKED|4 oz Burger Patty,1.0,PTN,NaN,1.0,PTN
7,P-51508,COOKED|Bacon|(2) Slice,2.0,slice,NaN,2.0,slice
8,P-37297,COOKED|Poached Egg,1.0,each,NaN,1.0,each
9,P-60812,COOKED|Tart Shell,1.0,each,NaN,1.0,each


In [60]:
# Filter out preps with nonstandard uom but have information already
Manual_PrepU = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Preps_UpdateUom.csv"))

col_names = list(Preps_Nonstd.columns.values)
Preps_Nonstd_na = []

for index, row in Preps_Nonstd.iterrows():
    PrepId = Preps_Nonstd.loc[index,'PrepId']
    if PrepId not in Manual_PrepU['PrepId'].values:
        Dict = {}
        Dict.update(dict(row))
        Preps_Nonstd_na.append(Dict)

Preps_Nonstd = pd.DataFrame(Preps_Nonstd_na, columns = col_names)
Preps_Nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-72899,Crusted|Halibut|LM,50.0,PTN,NaN,50.0,PTN
1,P-72896,Pistachio|Breaded|Tofu,100.0,PTN,NaN,100.0,PTN
2,P-72885,Risotto|Cake|LM,45.0,PTN,NaN,45.0,PTN


In [61]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_NonstdUom.csv")
Preps_Nonstd.to_csv(path, index = False, header = True)

In [62]:
update_prep = pd.read_csv("data/cleaning/update/Preps_UpdateUom.csv")
update_prep

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.0,each,PREP,84.0,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.0,ea,PREP,24050.0,g
2,P-51992,YIELD|Bread|Sourdough 5/8,36.0,slice,NaN,1620.0,g
3,P-26234,BATCH|Roasted Garlic Bread,16.0,ea,PREP,1280.0,g
4,P-26170,GRILLED|NaanBread,1.0,ea,PREP,125.0,g
...,...,...,...,...,...,...,...
470,P-73664,PREP|SMTHY|PressoEspresso|AAD,2.0,ea,NaN,582.5,g
471,P-38874,YIELD|Cod|85gram loin|CAL,1.0,piece,NaN,85.0,g
472,P-72899,Crusted|Halibut|LM,50.0,PTN,NaN,5250.0,g
473,P-72896,Pistachio|Breaded|Tofu,100.0,PTN,NaN,12200.0,g


***

## New Items

In [63]:
# Load current Items List with assigned Emission Factors Category ID
Items_Assigned = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv"))
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-10869,1,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
2,I-7064,1,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
3,I-37005,1,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT
4,I-37002,1,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT


In [64]:
Items_Assigned.shape

(2838, 8)

In [65]:
Items_Assigned["InventoryGroup"].unique()

array(['MEAT', 'DELI & PREPARED MEAT', 'FOOD - GROCERY', 'POULTRY',
       'PRODUCTION FOOD', 'DAIRY', 'BAKING-RAW INGREDIENTS', 'PRODUCE',
       'ICECREAM/NOVELTY/GELATO', 'SEAFOOD', 'CANDIES SNACKS', 'MISC.',
       'MM Grocery Cereal', 'BAKED GOODS', 'MM Grocery Chill', 'PREP',
       'BREAD', 'MM BAKERY', 'ZDONT USE FOOD', 'SPICES', 'FAIR TRADE',
       'MM Snack Ethnic', 'HM FROZEN SINGLE SERVE',
       'CHIPS, PRETZELS, NUTS', 'ZDONT USE FROZEN', 'BEVERAGE',
       'PACKAGED BEER, CIDER', 'LIQUOR', 'ALCOHOL - FOOD',
       'MM CANDY Chocolate ', 'PREGEL - ALL ITEMS',
       'HM SWEET & SAVOURY SNACKS', 'UCARE',
       'HM CHILL SINGLE SERVE BEVERAGE', 'JERKY, PEPPERONI', 'SUSHI',
       'MM CANDY Bulk', 'MM CANDY Gum & Mints', 'MM Snack Single Bars',
       'MERCHANDISE', 'MM Frozen Grocery', 'MM Frozen HMR', 'MM Grocery',
       'MM Grocery Coffee & Tea', 'MM Grocery Ethnic', 'MM Grocery Soup',
       'MM MERCH OTC', 'MM Snack Jerky', 'ZDONT USE CANDY - GUM & MINTS',
       'DI

In [66]:
Items_Assigned["CategoryID"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 56,
       57, 58, 59, 60, 61, 63, 15])

In [67]:
Items_Assigned[Items_Assigned["CategoryID"] == 55]

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
1287,I-11794,55,YERBA MATE REVEL BERRY 458ML,12.0,each,1.0,each,BEVERAGE
1288,I-1683,55,WATER VITAMIN ZERO XOXO,12.0,591ML,1.0,591ML,BEVERAGE
1289,I-2955,55,WATER SPARKLING SAN PELLEG,24.0,PTN,1.0,PTN,BEVERAGE
1290,I-52773,55,WATER SMART SPARKLING RASPB,12.0,each,1.0,each,BEVERAGE
1291,I-5983,55,WATER FOR RECIPES,1.0,L,1.0,L,BEVERAGE
...,...,...,...,...,...,...,...,...
2803,I-13795,55,STEAZ SUPERFRUIT ICED TEAZ,12.0,CT,1.0,CT,BEVERAGE
2817,I-45754,55,WATER ACTIVE LEMON LIME,12.0,450ML,1.0,450ML,BEVERAGE
2818,I-45755,55,WATER ACTIVE ORANGE MANGO,12.0,450ML,1.0,450ML,BEVERAGE
2819,I-45756,55,WATER ACTIVE STRAWB BLK CHERRY,12.0,450ML,1.0,450ML,BEVERAGE


### Get the List of New Items

In [68]:
# Filter new items by itemID that are not in the database and output them in a dataframe
col_names = list(Items.columns.values)
New_Items_List = []

for index, row in Items.iterrows():
    ItemId = Items.loc[index,'ItemId']
    if ItemId not in Items_Assigned['ItemId'].values:
        Dict = {}
        Dict.update(dict(row))
        New_Items_List.append(Dict)

New_Items = pd.DataFrame(New_Items_List, columns = col_names)

In [69]:
New_Items.insert(1, "CategoryID", '')
New_Items

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-72463,,BEAN BLACK*READY TO EAT,6.000,pak,4.00,lb,FOOD - GROCERY
1,I-72464,,BEANS GARBANZO*READY TO EAT,6.000,pak,4.00,lb,FOOD - GROCERY
2,I-72753,,BRIOCHE BREAD BRAIDED SLCD,7.000,LOAF,10.00,slice,BREAD
3,I-73416,,BROTH SHIO GARLIC PACKS,50.000,ea,1.00,ea,FOOD - GROCERY
4,I-73415,,BROTH TAN TAN ORGANIC,3.000,JUG,2.50,L,FOOD - GROCERY
5,I-70023,,BUN BRIOCHE*HOTDOG 6 INCH,72.000,each,1.00,each,BREAD
6,I-72414,,BURGER LIONS MANE,1.000,cs,1.00,cs,FOOD - GROCERY
7,I-69728,,BURRITO SPICY BEAN VEGAN,1.000,ea,1.00,ea,FOOD - GROCERY
8,I-71937,,CHEESE CHED*WHT OLD SHRED,4.000,bag,2.50,Kg,DAIRY
9,I-02341,,CHEESE FETA*11KG,1.000,pail,11.00,Kg,DAIRY


In [70]:
New_Items.shape

(45, 8)

In [71]:
# Store the list of new items into .csv file
# If New_Items is not empty then we convert it to a csv file. 
if not New_Items.empty:
    path = os.path.join(os.getcwd(), "data", "mapping", "new items", str(datetime.date(datetime.now()))+"_New_Items.csv")
    New_Items.to_csv(path, index = False, header = True)

In [72]:
# file = pd.read_csv("data/mapping/new items/2022-11-01_New_Items.csv")
# file.to_excel("2022-11-01_New_Items.xlsx",index = None, header=True)
# if not New_Items.empty:
#     file = pd.read_csv("C:/Users/ENTER_HERE/CFFS-S23/CFFS-22-23/data/mapping/new items/"+ str(datetime.date(datetime.now()))+"_New_Items.csv")
#     file.to_excel(str(datetime.date(datetime.now()))+"_New_Items.xlsx",index = None, header=True)

In [67]:
# file2 = pd.read_excel("data/mapping/new items added/New_Items_Added_11.xlsx")
# file2.to_csv("data/mapping/new items added/New_Items_Added_11.csv", index=False)

file2 = pd.read_excel("data/mapping/new items added/New_Items_2023/New_Items_Added_2023_08_01.xlsx")
file2.to_csv("data/mapping/new items added/New_Items_2023/New_Items_Added_2023_08_01.csv", index=False)

***
## Data Summary

In [73]:
datasum = pd.DataFrame([New_Items.shape, Preps_Nonstd.shape, Items_Nonstd.shape],
                       columns = ['count', 'columns'], 
                       index = ['New_Items', 'Preps_Nonstd', 'Items_Nonstd'])
datasum

,count,columns
New_Items,45,8
Preps_Nonstd,3,7
Items_Nonstd,62,7


In [74]:
print(New_Items.columns)

Index(['ItemId', 'CategoryID', 'Description', 'CaseQty', 'CaseUOM', 'PakQty',
       'PakUOM', 'InventoryGroup'],
      dtype='object')
